In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium import LayerControl
import geopandas as gpd

In [2]:
data = pd.read_excel("files/unidades.xlsx")
data = data[['NOMBRE DE LA UNIDAD', 'LATITUD', 'LONGITUD']]
data['LATITUD'] = pd.to_numeric(data['LATITUD'], errors='coerce')
data['LONGITUD'] = pd.to_numeric(data['LONGITUD'], errors='coerce')
data = data.dropna(subset=['LATITUD', 'LONGITUD'])

hosp = pd.read_excel("files/hospitales.xlsx")
df_hospitales = pd.DataFrame(hosp, columns=['NOMBRE DE LA UNIDAD', 'LATITUD', 'LONGITUD'])
df_hospitales['LATITUD'] = pd.to_numeric(df_hospitales['LATITUD'], errors='coerce')
df_hospitales['LONGITUD'] = pd.to_numeric(df_hospitales['LONGITUD'], errors='coerce')
df_hospitales = df_hospitales.dropna(subset=['LATITUD', 'LONGITUD'])
print(df_hospitales)

                                  NOMBRE DE LA UNIDAD    LATITUD   LONGITUD
0                            HOSPITAL GENERAL ACTOPAN  20.262789 -98.939827
1                         HOSPITAL INTEGRAL ATLAPEXCO  21.021042 -98.348585
2                          HOSPITAL GENERAL HUICHAPAN  20.375713 -99.654174
3   HOSPITAL GENERAL DEL VALLE DEL MEZQUITAL IXMIQ...  20.433787 -99.155446
4                            HOSPITAL INTEGRAL JACALA  21.010408 -99.194832
5                       HOSPITAL INTEGRAL CINTA LARGA  20.189068 -99.224940
6                            HOSPITAL GENERAL PACHUCA  20.113760 -98.722855
7                     HOSPITAL REGIONAL OTOMI TEPEHUA  20.395194 -98.203421
8                                    HOSPITAL ZIMAPÁN  20.690990 -99.333370
9                           HOSPITAL MATERNO INFANTIL  20.117775 -98.729856
10                    HOSPITAL INTEGRAL DE TLANCHINOL  20.998835 -98.653302
11                           HOSPITAL GENERAL DE APAN  19.713632 -98.465096
12          

In [3]:


try:
    gdf = gpd.read_file("files/muni_2018gw/muni_2018gw.shp", encoding='utf-8')
except UnicodeDecodeError:
    gdf = gpd.read_file("files/muni_2018gw/muni_2018gw.shp", encoding='latin-1')

# SELECCIONAR SOLO HIDALGO
hgo = gdf[gdf['NOM_ENT'] == 'Hidalgo']

# Crear el mapa centrado en la ubicación media de los datos
mapa = folium.Map(location=[data['LATITUD'].mean(), data['LONGITUD'].mean()], zoom_start=10)

# Añadir capa de clúster
marker_cluster = MarkerCluster().add_to(mapa)

# Añadir puntos personalizados y círculos al mapa dentro de la capa de clúster
for _, row in data.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=folium.Popup(
            f"<div style='text-align: center;'>"
            f"<h4 style='color: #006847; margin: 5px 0;'>🏥 {row['NOMBRE DE LA UNIDAD']}</h4>"
            f"<p style='margin: 3px 0; color: #666; font-size: 11px;'>"
            f"📍 Lat: {row['LATITUD']:.4f}, Lon: {row['LONGITUD']:.4f}</p>"
            f"<p style='margin: 5px 0; color: #006847; font-weight: bold;'>IMSS Bienestar</p>"
            f"</div>",
            max_width=350
        ),
        icon=folium.Icon(
            color='white',
            icon_color='#006847',
            icon='user-md',
            prefix='fa'
        ),
        tooltip=folium.Tooltip(
            f"🏥 {row['NOMBRE DE LA UNIDAD'][:50]}{'...' if len(row['NOMBRE DE LA UNIDAD']) > 50 else ''}",
            style="background-color: white; color: #333; border: 2px solid #006847; font-size: 12px; padding: 8px; border-radius: 5px;"
        )
    ).add_to(marker_cluster)
    
    folium.Circle(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=500,  # en metros, puedes ajustar según lo necesites
        color='#ADB5BD',
        fill=True,
        fill_color='#DEE2E6'
    ).add_to(mapa)

# Añadir hospitales (segundo nivel de atención) con marcadores diferenciados
cluster_hospitales = MarkerCluster(name='Hospitales').add_to(mapa)
for _, hospital in df_hospitales.iterrows():
    folium.Marker(
        location=[hospital['LATITUD'], hospital['LONGITUD']],
        popup=folium.Popup(
            f"<div style='text-align: center;'>"
            f"<h4 style='color: #B22222; margin: 5px 0;'>🏥 {hospital['NOMBRE DE LA UNIDAD']}</h4>"
            f"<p style='margin: 3px 0; color: #666;'><strong>HOSPITAL - 2° Nivel</strong></p>"
            f"<p style='margin: 3px 0; color: #666; font-size: 11px;'>"
            f"📍 Lat: {hospital['LATITUD']:.4f}, Lon: {hospital['LONGITUD']:.4f}</p>"
            f"<p style='margin: 5px 0; color: #B22222; font-weight: bold;'>Segundo Nivel de Atención</p>"
            f"</div>",
            max_width=400
        ),
        icon=folium.Icon(
            color='red',
            icon_color='white',
            icon='plus-square',
            prefix='fa'
        ),
        tooltip=folium.Tooltip(
            f"🏥 HOSPITAL: {hospital['NOMBRE DE LA UNIDAD'][:45]}{'...' if len(hospital['NOMBRE DE LA UNIDAD']) > 45 else ''}",
            style="background-color: white; color: #333; border: 2px solid #B22222; font-size: 12px; padding: 8px; border-radius: 5px;"
        )
    ).add_to(cluster_hospitales)
    
    # Círculo de cobertura más grande para hospitales
    folium.Circle(
        location=[hospital['LATITUD'], hospital['LONGITUD']],
        radius=1000,  # 1km para hospitales (mayor cobertura)
        color='#B22222',
        fill=True,
        fill_color='#FFB6C1',
        fillOpacity=0.3,
        opacity=0.6,
        weight=2
    ).add_to(mapa)

# Listas de municipios para cada jurisdicción
municipios_dict = {
    'Jurisdicción Tula': ['Ajacuba','Atitalaquia','Atotonilco de Tula','Tula de Allende','Tepeji del Río de Ocampo','Tepetitlán','Tetepango','Tezontepec de Aldama','Tlahuelilpan','Tlaxcoapan'],
    'Jurisdicción Tulancingo': ['Acatlán','Acaxochitlán','Agua Blanca de Iturbide','Cuautepec de Hinojosa','Huehuetla','Metepec','San Bartolo Tutotepec','Santiago Tulantepec de Lugo Guerrero','Singuilucan','Tenango de Doria','Tulancingo de Bravo'],
    'Jurisdicción Pachuca': ['Pachuca de Soto','San Agustín Tlaxiaca','Zapotlán de Juárez'],
    'Jurisdicción Huejutla': ['Atlapexco','Calnali','Huautla','Huazalingo','Huejutla de Reyes','Jaltocán','Lolotla','San Felipe Orizatlán','Tlanchinol','Xochiatipan','Yahualica'],
    'Jurisdicción Mineral': ['Epazoyucan','Mineral de la Reforma','Mineral del Chico','Omitlán de Juárez','Mineral del Monte','Huasca de Ocampo'],
    'Jurisdicción Tizayuca': ['Villa de Tezontepec','Tolcayuca','Tizayuca','Zempoala'],
    'Jurisdicción Actopan': ['Actopan','El Arenal','Francisco I. Madero','Mixquiahuala de Juárez','Progreso de Obregón','San Salvador','Santiago de Anaya'],
    'Jurisdicción Ixmiquilpan': ['Alfajayucan','Cardonal','Chilcuautla','Ixmiquilpan','Nicolás Flores','Tasquillo','Zimapán'],
    'Jurisdicción Molango': ['Atotonilco el Grande','Eloxochitlán','Juárez Hidalgo','Metztitlán','San Agustín Metzquititlán','Molango de Escamilla','Tepehuacán de Guerrero','Tianguistengo','Tlahuiltepa','Xochicoatlán','Zacualtipán de Ángeles'],
    'Jurisdicción Apan': ['Almoloya', 'Apan','Emiliano Zapata','Tepeapulco','Tlanalapa'],
    'Jurisdicción Huichapan': ['Chapantongo','Huichapan','Nopala de Villagrán','Tecozautla'],
    'Jurisdicción Jacala': ['Chapulhuacán','Jacala de Ledezma','La Misión','Pacula','Pisaflores']
}

# Colores para cada jurisdicción
colores = {
    'Jurisdicción Tula': '#52796F', 'Jurisdicción Tulancingo': '#3f37c9', 'Jurisdicción Pachuca': '#4cc9f0', 'Jurisdicción Huejutla': '#ffd60a', 'Jurisdicción Mineral': '#343A40',
    'Jurisdicción Tizayuca': '#354F52', 'Jurisdicción Actopan': '#f28482', 'Jurisdicción Ixmiquilpan': '#d80032', 'Jurisdicción Molango': '#db7c26', 'Jurisdicción Apan': '#274c77',
    'Jurisdicción Huichapan': '#eddea4', 'Jurisdicción Jacala': '#00afb9'
}

# Agrupar municipios y añadir capas de cada jurisdicción
for nombre_jurisdiccion, municipios in municipios_dict.items():
    grupo = folium.FeatureGroup(name=nombre_jurisdiccion)
    for municipio in municipios:
        folium.GeoJson(
            hgo[hgo['NOM_MUN'] == municipio],
            style_function=lambda feature, color=colores[nombre_jurisdiccion]: {
                'fillColor': color,
                'color': color,
                'weight': 2,
                'fillOpacity': 0.5,
                'opacity': 0.7,
                'lineCap': 'round',
                'lineJoin': 'round'
            }
        ).add_to(grupo)
    grupo.add_to(mapa)

# Añadir controles de capas
folium.LayerControl().add_to(mapa)

title_html = ''' <h3 align="center" style="font-size:20px"><b>Mapa de Unidades de salud Hidalgo</b></h3> '''
mapa.get_root().html.add_child(folium.Element(title_html))
# Guardar el mapa en un archivo HTML
mapa.save('files/mapa_localidades.html')

# Mostrar mensaje de éxito
print("El mapa se ha guardado exitosamente en 'mapa_localidades.html'")


El mapa se ha guardado exitosamente en 'mapa_localidades.html'


In [ ]:
# MAPA ESPECÍFICO PARA MUNICIPIOS DE IMSS BIENESTAR
# Municipios seleccionados para IMSS Bienestar
municipios_imss_bienestar = [
    'Tula de Allende', 'Tepeji del Río de Ocampo', 'Atitalaquia', 'Tlaxcoapan', 
    'Atotonilco de Tula', 'Tetepango', 'Tepetitlán', 'Tlahuelilpan', 
    'Tezontepec de Aldama', 'Chapantongo', 'Nopala de Villagrán'
]

# Filtrar el GeoDataFrame para incluir solo los municipios de IMSS Bienestar
hgo_imss = hgo[hgo['NOM_MUN'].isin(municipios_imss_bienestar)]

# Crear GeoDataFrame con las unidades de salud
gdf_unidades = gpd.GeoDataFrame(
    data, 
    geometry=gpd.points_from_xy(data['LONGITUD'], data['LATITUD']),
    crs='EPSG:4326'
)

# Asegurar que ambos GeoDataFrames tienen el mismo CRS
hgo_imss = hgo_imss.to_crs('EPSG:4326')

# Filtrar unidades que están dentro de los municipios de IMSS Bienestar
unidades_en_municipios = gpd.sjoin(gdf_unidades, hgo_imss, how='inner', predicate='within')

# Crear DataFrame filtrado solo con las unidades dentro de los municipios objetivo
data_imss = unidades_en_municipios[['NOMBRE DE LA UNIDAD', 'LATITUD', 'LONGITUD']].copy()

print(f"🔍 Unidades totales: {len(data)}")
print(f"📍 Unidades en municipios IMSS Bienestar: {len(data_imss)}")
print(f"📋 Municipios con unidades: {sorted(unidades_en_municipios['NOM_MUN'].unique())}")

# Crear un nuevo mapa centrado en la zona de estos municipios
if len(data_imss) > 0:
    mapa_imss = folium.Map(location=[data_imss['LATITUD'].mean(), data_imss['LONGITUD'].mean()], zoom_start=11)
else:
    mapa_imss = folium.Map(location=[20.05, -99.35], zoom_start=10)

# Color oficial de IMSS Bienestar - Verde institucional
color_imss_bienestar = '#006847'  # Verde IMSS Bienestar
color_imss_secundario = '#4A9B3A'  # Verde complementario

# Añadir los polígonos de los municipios de IMSS Bienestar
folium.GeoJson(
    hgo_imss,
    style_function=lambda feature: {
        'fillColor': color_imss_bienestar,
        'color': color_imss_bienestar,
        'weight': 3,
        'fillOpacity': 0.6,
        'opacity': 0.9,
        'lineCap': 'round',
        'lineJoin': 'round'
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NOM_MUN'],
        aliases=['Municipio:'],
        style="background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"
    )
).add_to(mapa_imss)

# Añadir SOLO las unidades de salud que están dentro de los municipios objetivo (SIN CLUSTER)
for _, row in data_imss.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=folium.Popup(
            f"<div style='text-align: center;'>"
            f"<h4 style='color: {color_imss_bienestar}; margin: 5px 0;'>🏥 {row['NOMBRE DE LA UNIDAD']}</h4>"
            f"<p style='margin: 3px 0; color: #666; font-size: 11px;'>"
            f"📍 Lat: {row['LATITUD']:.4f}, Lon: {row['LONGITUD']:.4f}</p>"
            f"<p style='margin: 5px 0; color: {color_imss_bienestar}; font-weight: bold;'>IMSS Bienestar</p>"
            f"</div>",
            max_width=400
        ),
        icon=folium.Icon(
            color='white',
            icon_color=color_imss_bienestar,
            icon='user-md',
            prefix='fa'
        ),
        tooltip=folium.Tooltip(
            f"🏥 {row['NOMBRE DE LA UNIDAD'][:50]}{'...' if len(row['NOMBRE DE LA UNIDAD']) > 50 else ''}",
            style=f"background-color: white; color: #333; border: 2px solid {color_imss_bienestar}; font-size: 12px; padding: 8px; border-radius: 5px;"
        )
    ).add_to(mapa_imss)
    
    # Círculos de cobertura con colores IMSS Bienestar
    folium.Circle(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=500,  # 500 metros de radio de cobertura
        color=color_imss_bienestar,
        fill=True,
        fill_color=color_imss_secundario,
        fillOpacity=0.3,
        opacity=0.7,
        weight=2
    ).add_to(mapa_imss)

# Filtrar hospitales en la zona de IMSS Bienestar
gdf_hospitales = gpd.GeoDataFrame(
    df_hospitales, 
    geometry=gpd.points_from_xy(df_hospitales['LONGITUD'], df_hospitales['LATITUD']),
    crs='EPSG:4326'
)
hospitales_en_zona = gpd.sjoin(gdf_hospitales, hgo_imss, how='inner', predicate='within')

# Agregar hospitales (segundo nivel) en la zona IMSS Bienestar
for _, hospital in hospitales_en_zona.iterrows():
    folium.Marker(
        location=[hospital['LATITUD'], hospital['LONGITUD']],
        popup=folium.Popup(
            f"<div style='text-align: center;'>"
            f"<h4 style='color: #B22222; margin: 5px 0;'>🏥 {hospital['NOMBRE DE LA UNIDAD']}</h4>"
            f"<p style='margin: 3px 0; color: #666;'><strong>HOSPITAL - 2° Nivel</strong></p>"
            f"<p style='margin: 3px 0; color: #666; font-size: 11px;'>"
            f"📍 Lat: {hospital['LATITUD']:.4f}, Lon: {hospital['LONGITUD']:.4f}</p>"
            f"<p style='margin: 5px 0; color: #B22222; font-weight: bold;'>Segundo Nivel de Atención</p>"
            f"</div>",
            max_width=400
        ),
        icon=folium.Icon(
            color='red',
            icon_color='white',
            icon='plus-square',
            prefix='fa'
        ),
        tooltip=folium.Tooltip(
            f"🏥 HOSPITAL: {hospital['NOMBRE DE LA UNIDAD'][:45]}{'...' if len(hospital['NOMBRE DE LA UNIDAD']) > 45 else ''}",
            style="background-color: white; color: #333; border: 2px solid #B22222; font-size: 12px; padding: 8px; border-radius: 5px;"
        )
    ).add_to(mapa_imss)
    
    # Círculo de cobertura para hospitales (mayor radio)
    folium.Circle(
        location=[hospital['LATITUD'], hospital['LONGITUD']],
        radius=1000,  # 1km para hospitales
        color='#B22222',
        fill=True,
        fill_color='#FFB6C1',
        fillOpacity=0.3,
        opacity=0.6,
        weight=2
    ).add_to(mapa_imss)

print(f"🏥 Hospitales en zona IMSS Bienestar: {len(hospitales_en_zona)}")

# Añadir título específico para IMSS Bienestar
title_html_imss = '''
<h3 align="center" style="font-size:22px; color: #006847; font-weight: bold;">
    <b>MAPA DE UNIDADES IMSS BIENESTAR</b><br>
    <span style="font-size:16px; color: #4A9B3A;">Región Tula - Tepeji del Río</span>
</h3>
'''
mapa_imss.get_root().html.add_child(folium.Element(title_html_imss))

# Agregar leyenda personalizada
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
<p><b style="color: #006847;">IMSS Bienestar</b></p>
<p><i class="fa fa-plus" style="color: green;"></i> Unidades de Salud</p>
<p><span style="color: #006847;">🏥</span>  Hospitales (2do Nivel)</p>  
<p><span style="background-color: #006847; color: white; padding: 2px 6px;">█</span> Municipios Objetivo</p>
</div>
'''
mapa_imss.get_root().html.add_child(folium.Element(legend_html))

# Guardar el mapa específico de IMSS Bienestar
mapa_imss.save('files/mapa_imss_bienestar.html')

print("✅ Mapa de IMSS Bienestar creado exitosamente!")
print(f"📊 Municipios incluidos: {len(municipios_imss_bienestar)}")
print(f"📍 Unidades de salud mostradas: {len(data_imss)}")
print("💾 Guardado como: 'files/mapa_imss_bienestar.html'")
print(f"🎨 Color principal: {color_imss_bienestar} (Verde IMSS Bienestar)")

🔍 Unidades totales: 523
📍 Unidades en municipios IMSS Bienestar: 69
📋 Municipios con unidades: ['Atitalaquia', 'Atotonilco de Tula', 'Chapantongo', 'Nopala de Villagrán', 'Tepeji del Río de Ocampo', 'Tepetitlán', 'Tetepango', 'Tezontepec de Aldama', 'Tlahuelilpan', 'Tlaxcoapan', 'Tula de Allende']
🏥 Hospitales en zona IMSS Bienestar: 2
✅ Mapa de IMSS Bienestar creado exitosamente!
📊 Municipios incluidos: 11
📍 Unidades de salud mostradas: 69
💾 Guardado como: 'files/mapa_imss_bienestar.html'
🎨 Color principal: #006847 (Verde IMSS Bienestar)
✅ Mapa de IMSS Bienestar creado exitosamente!
📊 Municipios incluidos: 11
📍 Unidades de salud mostradas: 69
💾 Guardado como: 'files/mapa_imss_bienestar.html'
🎨 Color principal: #006847 (Verde IMSS Bienestar)


In [5]:
# GENERAR MAPAS INDIVIDUALES POR MUNICIPIO DE IMSS BIENESTAR
import os

# Crear directorio para mapas individuales si no existe
os.makedirs('files/mapas_municipios', exist_ok=True)

# Colores variados para cada municipio (manteniendo la paleta IMSS Bienestar)
colores_municipios = {
    'Tula de Allende': '#006847',        # Verde principal IMSS
    'Tepeji del Río de Ocampo': '#4A9B3A',   # Verde complementario
    'Atitalaquia': '#2E7D4F',           # Verde oscuro
    'Tlaxcoapan': '#5FAD56',            # Verde claro
    'Atotonilco de Tula': '#1B5E3F',    # Verde muy oscuro
    'Tetepango': '#7BC159',             # Verde lima
    'Tepetitlán': '#0A4A2A',            # Verde institucional oscuro
    'Tlahuelilpan': '#94D773',          # Verde pastel
    'Tezontepec de Aldama': '#006847',  # Verde medio
    'Chapantongo': '#B8E6A0',           # Verde muy claro
    'Nopala de Villagrán': '#2F5233'    # Verde profundo
}

mapas_creados = []
total_unidades_municipios = 0

print("🗺️ GENERANDO MAPAS INDIVIDUALES POR MUNICIPIO")
print("=" * 60)

for municipio in municipios_imss_bienestar:
    # Filtrar datos del municipio específico
    municipio_gdf = hgo_imss[hgo_imss['NOM_MUN'] == municipio]
    unidades_municipio = unidades_en_municipios[unidades_en_municipios['NOM_MUN'] == municipio]
    
    if len(municipio_gdf) > 0:
        # Obtener el centroide del municipio para el centro del mapa
        centroide = municipio_gdf.geometry.centroid.iloc[0]
        centro_lat = centroide.y
        centro_lon = centroide.x
        
        # Crear mapa individual para el municipio
        mapa_municipio = folium.Map(
            location=[centro_lat, centro_lon], 
            zoom_start=13,
            tiles='OpenStreetMap'
        )
        
        # Color específico del municipio
        color_municipio = colores_municipios.get(municipio, '#006847')
        color_secundario = '#4A9B3A'
        
        # Añadir polígono del municipio
        folium.GeoJson(
            municipio_gdf,
            style_function=lambda feature, color=color_municipio: {
                'fillColor': color,
                'color': color,
                'weight': 4,
                'fillOpacity': 0.4,
                'opacity': 0.8,
                'lineCap': 'round',
                'lineJoin': 'round'
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['NOM_MUN'],
                aliases=['Municipio:'],
                style="background-color: white; color: #333333; font-family: arial; font-size: 14px; padding: 10px;"
            )
        ).add_to(mapa_municipio)
        
        # Añadir unidades de salud del municipio
        unidades_count = 0
        if len(unidades_municipio) > 0:
            for _, unidad in unidades_municipio.iterrows():
                # Marcador personalizado de la unidad con icono médico
                folium.Marker(
                    location=[unidad['LATITUD'], unidad['LONGITUD']],
                    popup=folium.Popup(
                        f"<div style='text-align: center;'>"
                        f"<h4 style='color: {color_municipio}; margin: 5px 0;'>🏥 {unidad['NOMBRE DE LA UNIDAD']}</h4>"
                        f"<p style='margin: 3px 0; color: #666;'><strong>Municipio:</strong> {municipio}</p>"
                        f"<p style='margin: 3px 0; color: #666; font-size: 11px;'>"
                        f"📍 Lat: {unidad['LATITUD']:.4f}, Lon: {unidad['LONGITUD']:.4f}</p>"
                        f"<p style='margin: 5px 0; color: {color_municipio}; font-weight: bold;'>IMSS Bienestar</p>"
                        f"</div>",
                        max_width=400
                    ),
                    icon=folium.Icon(
                        color='white',
                        icon_color=color_municipio,
                        icon='user-md',
                        prefix='fa'
                    ),
                    tooltip=folium.Tooltip(
                        f"🏥 {unidad['NOMBRE DE LA UNIDAD'][:50]}{'...' if len(unidad['NOMBRE DE LA UNIDAD']) > 50 else ''}",
                        style="background-color: white; color: #333; border: 2px solid " + color_municipio + "; font-size: 12px; padding: 8px; border-radius: 5px;"
                    )
                ).add_to(mapa_municipio)
                
                # Círculo de cobertura
                folium.Circle(
                    location=[unidad['LATITUD'], unidad['LONGITUD']],
                    radius=500,
                    color=color_municipio,
                    fill=True,
                    fill_color=color_secundario,
                    fillOpacity=0.2,
                    opacity=0.6,
                    weight=2,
                    tooltip="Radio de cobertura: 500m"
                ).add_to(mapa_municipio)
                
                unidades_count += 1
        
        # Agregar hospitales en el municipio
        hospitales_municipio = hospitales_en_zona[hospitales_en_zona['NOM_MUN'] == municipio] if 'hospitales_en_zona' in locals() else []
        hospitales_count = 0
        
        if len(hospitales_municipio) > 0:
            for _, hospital in hospitales_municipio.iterrows():
                folium.Marker(
                    location=[hospital['LATITUD'], hospital['LONGITUD']],
                    popup=folium.Popup(
                        f"<div style='text-align: center;'>"
                        f"<h4 style='color: #B22222; margin: 5px 0;'>🏥 {hospital['NOMBRE DE LA UNIDAD']}</h4>"
                        f"<p style='margin: 3px 0; color: #666;'><strong>HOSPITAL - 2° Nivel</strong></p>"
                        f"<p style='margin: 3px 0; color: #666;'><strong>Municipio:</strong> {municipio}</p>"
                        f"<p style='margin: 3px 0; color: #666; font-size: 11px;'>"
                        f"📍 Lat: {hospital['LATITUD']:.4f}, Lon: {hospital['LONGITUD']:.4f}</p>"
                        f"<p style='margin: 5px 0; color: #B22222; font-weight: bold;'>Segundo Nivel de Atención</p>"
                        f"</div>",
                        max_width=400
                    ),
                    icon=folium.Icon(
                        color='red',
                        icon_color='white',
                        icon='plus-square',
                        prefix='fa'
                    ),
                    tooltip=folium.Tooltip(
                        f"🏥 HOSPITAL: {hospital['NOMBRE DE LA UNIDAD'][:45]}{'...' if len(hospital['NOMBRE DE LA UNIDAD']) > 45 else ''}",
                        style="background-color: white; color: #333; border: 2px solid #B22222; font-size: 12px; padding: 8px; border-radius: 5px;"
                    )
                ).add_to(mapa_municipio)
                
                # Círculo de cobertura más grande para hospitales
                folium.Circle(
                    location=[hospital['LATITUD'], hospital['LONGITUD']],
                    radius=1000,  # 1km para hospitales
                    color='#B22222',
                    fill=True,
                    fill_color='#FFB6C1',
                    fillOpacity=0.3,
                    opacity=0.6,
                    weight=2,
                    tooltip="Hospital - Radio de cobertura: 1000m"
                ).add_to(mapa_municipio)
                
                hospitales_count += 1
        
        # Título GRANDE personalizado para cada municipio con color institucional IMSS Bienestar
        titulo_municipio = f'''
        <div style="position: fixed; top: 0; left: 0; right: 0; z-index: 1000; 
                    background: #006847; 
                    color: white; padding: 20px 0; text-align: center; 
                    box-shadow: 0 4px 15px rgba(0,0,0,0.3);">
            <h1 style="margin: 0; font-size: 36px; font-weight: bold; text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
                {municipio.upper()}
            </h1>
            <h2 style="margin: 5px 0 0 0; font-size: 20px; font-weight: normal; opacity: 0.9;">
                UNIDADES DE SALUD IMSS BIENESTAR
            </h2>
            <p style="margin: 5px 0 0 0; font-size: 16px; opacity: 0.8;">
                📍 {unidades_count} Unidad{'es' if unidades_count != 1 else ''} de Salud • 🏥 {hospitales_count} Hospital{'es' if hospitales_count != 1 else ''}
            </p>
        </div>
        
        <style>
            body {{ margin-top: 120px !important; }}
            .leaflet-control-container .leaflet-top {{ top: 120px !important; }}
        </style>
        '''
        mapa_municipio.get_root().html.add_child(folium.Element(titulo_municipio))
        
        # Crear tabla lateral con lista de unidades y hospitales
        tabla_unidades = ""
        if len(unidades_municipio) > 0:
            tabla_unidades = "<h4 style='color: " + color_municipio + "; margin: 0 0 10px 0; border-bottom: 2px solid " + color_municipio + "; padding-bottom: 5px;'>📋 Unidades de Salud (1er Nivel)</h4>"
            for idx, unidad in unidades_municipio.iterrows():
                nombre_corto = unidad['NOMBRE DE LA UNIDAD']
                if len(nombre_corto) > 45:
                    nombre_corto = nombre_corto[:42] + "..."
                tabla_unidades += f'''
                <div style="margin: 8px 0; padding: 8px; background: #f8f9fa; border-radius: 5px; border-left: 3px solid {color_municipio};">
                    <strong style="color: #333; font-size: 12px; display: block;">👨‍⚕️ {nombre_corto}</strong>
                    <small style="color: #666; font-size: 10px;">📍 {unidad['LATITUD']:.4f}, {unidad['LONGITUD']:.4f}</small>
                </div>
                '''
        
        # Agregar hospitales a la tabla lateral
        tabla_hospitales = ""
        if hospitales_count > 0:
            tabla_hospitales = "<h4 style='color: #B22222; margin: 15px 0 10px 0; border-bottom: 2px solid #B22222; padding-bottom: 5px;'>🏥 Hospitales (2do Nivel)</h4>"
            for idx, hospital in hospitales_municipio.iterrows():
                nombre_corto = hospital['NOMBRE DE LA UNIDAD']
                if len(nombre_corto) > 45:
                    nombre_corto = nombre_corto[:42] + "..."
                tabla_hospitales += f'''
                <div style="margin: 8px 0; padding: 8px; background: #fff5f5; border-radius: 5px; border-left: 3px solid #B22222;">
                    <strong style="color: #333; font-size: 12px; display: block;">🏥 {nombre_corto}</strong>
                    <small style="color: #666; font-size: 10px;">📍 {hospital['LATITUD']:.4f}, {hospital['LONGITUD']:.4f}</small>
                </div>
                '''
        
        # Panel lateral con información completa
        panel_lateral = f'''
        <div style="position: fixed; top: 130px; right: 20px; width: 320px; max-height: calc(100vh - 150px); 
                    background-color: white; border: 2px solid {color_municipio}; z-index: 9999; 
                    font-size: 12px; padding: 15px; border-radius: 10px; 
                    box-shadow: 0 4px 20px rgba(0,0,0,0.2); overflow-y: auto;">
            
            <div style="text-align: center; margin-bottom: 15px; padding: 10px; background: {color_municipio}; color: white; border-radius: 8px; margin: -15px -15px 15px -15px;">
                <h3 style="margin: 0; font-size: 16px;">{municipio}</h3>
                <p style="margin: 5px 0 0 0; font-size: 12px; opacity: 0.9;">IMSS Bienestar</p>
            </div>
            
            <div style="margin-bottom: 15px; padding: 10px; background: #f0f8f0; border-radius: 6px;">
                <h4 style="margin: 0 0 8px 0; color: {color_municipio}; font-size: 14px;">📊 Resumen</h4>
                <p style="margin: 3px 0;"><strong>{unidades_count}</strong> Unidad{'es' if unidades_count != 1 else ''} de Salud (1er Nivel)</p>
                <p style="margin: 3px 0;"><strong style="color: #B22222;">{hospitales_count}</strong> Hospital{'es' if hospitales_count != 1 else ''} (2do Nivel)</p>
                <p style="margin: 3px 0;"><span style="background-color: {color_municipio}; color: white; padding: 1px 4px; border-radius: 3px; font-size: 10px;">█</span> Límites Municipales</p>
            </div>
            
            <div style="max-height: 400px; overflow-y: auto;">
                {tabla_unidades}
                {tabla_hospitales}
            </div>
            
            <div style="text-align: center; margin-top: 15px; padding: 8px; background: #f8f9fa; border-radius: 6px;">
                <small style="color: #666; font-size: 10px;">
                    <strong>IMSS Bienestar</strong><br>
                    Generado: {pd.Timestamp.now().strftime('%d/%m/%Y %H:%M')}
                </small>
            </div>
        </div>
        '''
        mapa_municipio.get_root().html.add_child(folium.Element(panel_lateral))
        
        # Nombre del archivo (limpiar caracteres especiales)
        nombre_archivo = municipio.lower().replace(' ', '_').replace('ó', 'o').replace('í', 'i').replace('ñ', 'n')
        archivo_mapa = f'files/mapas_municipios/mapa_{nombre_archivo}.html'
        
        # Guardar el mapa
        mapa_municipio.save(archivo_mapa)
        mapas_creados.append({'municipio': municipio, 'archivo': archivo_mapa, 'unidades': unidades_count})
        total_unidades_municipios += unidades_count
        
        print(f"✅ {municipio:<25} → {unidades_count:>2} unidades → {archivo_mapa}")

print("=" * 60)
print(f"📊 RESUMEN DE MAPAS CREADOS:")
print(f"   🗺️  Total de mapas: {len(mapas_creados)}")
print(f"   📍 Total de unidades: {total_unidades_municipios}")
print(f"   📁 Directorio: files/mapas_municipios/")
print(f"   🎨 Colores personalizados por municipio")

# Crear un archivo índice HTML para navegar entre mapas
indice_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Índice de Mapas - IMSS Bienestar</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; background-color: #f8f9fa; }
        .header { text-align: center; color: #006847; margin-bottom: 30px; }
        .municipio-card { 
            background: white; margin: 10px 0; padding: 15px; border-radius: 8px; 
            border-left: 4px solid #006847; box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: transform 0.2s;
        }
        .municipio-card:hover { transform: translateY(-2px); }
        .municipio-link { 
            text-decoration: none; color: #006847; font-weight: bold; font-size: 16px;
        }
        .unidades-count { color: #4A9B3A; font-size: 14px; margin-top: 5px; }
        .total-summary { 
            background: #006847; color: white; padding: 20px; border-radius: 8px; 
            text-align: center; margin: 20px 0;
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🗺️ MAPAS MUNICIPALES - IMSS BIENESTAR</h1>
        <p>Región Tula - Tepeji del Río</p>
    </div>
    
    <div class="total-summary">
        <h3>📊 Resumen General</h3>
        <p><strong>''' + str(len(mapas_creados)) + '''</strong> municipios • <strong>''' + str(total_unidades_municipios) + '''</strong> unidades de salud</p>
    </div>
    
    <div class="mapas-lista">
'''

for mapa_info in mapas_creados:
    nombre_archivo = mapa_info['archivo'].split('/')[-1]
    indice_html += f'''
        <div class="municipio-card">
            <a href="{nombre_archivo}" class="municipio-link">{mapa_info['municipio']}</a>
            <div class="unidades-count">📍 {mapa_info['unidades']} unidad{'es' if mapa_info['unidades'] != 1 else ''} de salud</div>
        </div>
    '''

indice_html += '''
    </div>
    
    <div style="text-align: center; margin-top: 30px; color: #666; font-size: 12px;">
        <p>Generado el ''' + str(pd.Timestamp.now().strftime('%d/%m/%Y %H:%M')) + ''' • IMSS Bienestar</p>
    </div>
</body>
</html>
'''

with open('files/mapas_municipios/indice.html', 'w', encoding='utf-8') as f:
    f.write(indice_html)

print(f"📋 Archivo índice creado: files/mapas_municipios/indice.html")

🗺️ GENERANDO MAPAS INDIVIDUALES POR MUNICIPIO
✅ Tula de Allende           → 18 unidades → files/mapas_municipios/mapa_tula_de_allende.html


/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project 

✅ Tepeji del Río de Ocampo  →  7 unidades → files/mapas_municipios/mapa_tepeji_del_rio_de_ocampo.html
✅ Atitalaquia               →  7 unidades → files/mapas_municipios/mapa_atitalaquia.html
✅ Tlaxcoapan                →  3 unidades → files/mapas_municipios/mapa_tlaxcoapan.html
✅ Atotonilco de Tula        →  4 unidades → files/mapas_municipios/mapa_atotonilco_de_tula.html
✅ Tetepango                 →  2 unidades → files/mapas_municipios/mapa_tetepango.html


/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project 

✅ Tepetitlán                →  4 unidades → files/mapas_municipios/mapa_tepetitlán.html
✅ Tlahuelilpan              →  4 unidades → files/mapas_municipios/mapa_tlahuelilpan.html
✅ Tezontepec de Aldama      → 11 unidades → files/mapas_municipios/mapa_tezontepec_de_aldama.html
✅ Chapantongo               →  4 unidades → files/mapas_municipios/mapa_chapantongo.html
✅ Nopala de Villagrán       →  5 unidades → files/mapas_municipios/mapa_nopala_de_villagrán.html
📊 RESUMEN DE MAPAS CREADOS:
   🗺️  Total de mapas: 11
   📍 Total de unidades: 69
   📁 Directorio: files/mapas_municipios/
   🎨 Colores personalizados por municipio
📋 Archivo índice creado: files/mapas_municipios/indice.html
✅ Nopala de Villagrán       →  5 unidades → files/mapas_municipios/mapa_nopala_de_villagrán.html
📊 RESUMEN DE MAPAS CREADOS:
   🗺️  Total de mapas: 11
   📍 Total de unidades: 69
   📁 Directorio: files/mapas_municipios/
   🎨 Colores personalizados por municipio
📋 Archivo índice creado: files/mapas_municipios/indic

/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
/tmp/ipykernel_6475/2696719802.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide = municipio_gdf.geometry.centroid.iloc[0]
